## Summary


The following data set if for an ecommerce company that sells tires.  The data set is from Google paid search queries related to Product Listing Ads (PLAs) which include Google Shopping queries. The search terms that include Client’s brand name have been excluded. Given this data, and any other information you want to add, what can you tell us? What information/visualizations can we share with the customer to help them position/re-position etc. How are people finding their products, and what can we do with this information?

Original Query Parameters:

Campaign Type: Google Shopping PLAs
Network: Google Search; Excluding all other Audience Extension Networks
Note: This file is based upon Search Terms (not Keywords), which are the actual queries that users typed in

Conversion Parameters (e.g., how is Conversions and Conversion Value calculated):

Includes the revenue generated from transactions that occur within 60 days of an ad click; multiple transactions within 60 days are counted
Conversion Values are modeled on a “Position-based attribution model”, which means that Google Ads allocates conversion value to ads based on: First Ad Click = 40%, Last Ad Click = 40%, All Middle Clicks = 20% (Linear Split)
As a result, Conversions and Conversion Values are not a perfect measure as we don’t know if the conversion value was due to a search query being the 1st click, last click, or a middle click

Calculations Not Included:
CV Gross Profit = Conversion Value * 20%
CV Gross Profit Less Ad Cost = (Conversion Value * 20% ) – Ad Cost
MCF Revenue = (Conversion Value * 50%)
MCF Gross Profit = ((Conversion Value * 50%) *20%) - https://www.ppchero.com/ultimate-guide-to-googles-multi-channel-funnels/
MCF Gross Profit Less Ad Cost = ((Conversion Value * 50%) * 20%) – Ad Cost

Single Click w/Measures

Columns:

     - Full Words
     - Impressions
     - Clicks (Drop after confirmation)
     - Measures
         - Tire Type (P, LT or none)
         - Tire Width (first 3 numbers in millimeters)
         - Aspect Ratio (2 digit number after "/", height equal to the % of tire's width.) 
         - Construction (If R is present then the layers run radially across the tire
         - Wheel Diameter (Size of wheel the tire is supposed to fit)
         
         - Unknowns
             - Load Index
             - Speed Rating
             - DOT Sumbol
             - Tire Identification Number
             - UTQG
      - Cost, Conversion_Value, Conversions, Conv_value_cost: Brought but not for model
[Source](https://www.goodyearautoservice.com/en-US/tire-basics/tire-size)
     
no clicks?
few clicks?

 - Impression: https://www.klipfolio.com/metrics/difference/ad-clicks-vs-ad-impressions#:~:text=Ad%20Clicks%2C%20or%20simply%20Clicks,to%20reach%20an%20online%20property.&text=Ad%20Impressions%20(IMPR)%20is%20a,screen%20within%20the%20publisher's%20network.
 
 - Impression Share: https://support.google.com/google-ads/answer/2497703?hl=en
  
 - Conversion: A conversion in Google Ads is when a user performs some specified action after clicking an ad or viewing a Display Network ad, such as purchasing a product, installing a mobile app, or signing up for an email list. Conversion tracking provides key insights into users' actions after viewing or clicking an ad.

## Imports & Such

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as ran
%matplotlib inline
import re
import random

In [74]:
# Imports
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re

In [75]:
df = pd.read_csv('./../G_ad_data/G_ad_clean.csv', index_col= False)
df_raw = pd.read_csv('./../G_ad_data/Search_terms_cleaned.csv')#.columns

C:\Users\Peridot of Earth\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [76]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,CV_gross_profit,CV_gross_profit_less_ad_cost,MCF_revenue,MCF_gross_profit,MCF_gross_profit_less_ad_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,0.000,-2.220,0.00,0.000,-2.220
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,0.000,-0.220,0.00,0.000,-0.220
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67,1.264,-2.516,3.16,0.632,-3.148
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.00,0.00,0.00,0.00,0.000,-0.170,0.00,0.000,-0.170
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,87.620,65.930,219.05,43.810,22.120


## Cleaning

In [77]:
df.drop(['CV_gross_profit', 'CV_gross_profit_less_ad_cost', 
         'MCF_revenue', 'MCF_gross_profit', 'MCF_gross_profit_less_ad_cost'], axis = 1, inplace = True)

In [78]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00
2,wild country at,3.78,0.22,39,17,0.07,6.32,0.43,1.67
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.00,0.00,0.00,0.00
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20


## Filtering to test

In [79]:
df = df[df['Clicks'] == 1]
print(df.shape)
df.head()


(1500697, 9)


,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,0.0,0.0
3,h w 100xt primex tires 7.50 16,0.17,0.17,1,1,0.0,0.0,0.0,0.0
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,0.0,0.0
9,5x by 30 r20 tire,0.52,0.52,1,1,0.0,0.0,0.0,0.0
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,0.0,0.0


In [80]:
# Isolating all potnetial Tire Measurement Numbersets
list_results = []
for row in df['Search_term']:
    result = re.findall(r'p?l?t?\d{3} \d{2}.?\d{0,3} ?r? ?\d{2}', row)#(r'[\d.r \/]+') #(r'[\d.rbyhpx ]+')
    
    list_inner = []
    for item in result:
        item = item.strip(' ')
        #if len(item) > 7 & len(item) < 12:
        list_inner.append(item)
    list_results.append(list_inner)

In [81]:
# Counting how many Measurements Numbersets are present
list_lists_count = []
for num in list_results:
    list_lists_count.append(len(num))

for num in range(11):
    print(num, list_lists_count.count(num))

0 849087
1 646386
2 5208
3 15
4 1
5 0
6 0
7 0
8 0
9 0
10 0


In [82]:
df['Tire_measures'] = list_results

In [83]:
df['list_counts'] = list_lists_count

In [84]:
df = df[df['list_counts'] == 1]
print(df.shape)
df.head()

(646386, 11)


,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Tire_measures,list_counts
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,0.0,0.0,[325 45r22],1
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,0.0,0.0,[235 65 17],1
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,0.0,0.0,[235 75 16],1
13,275 65 20 cooper discover pro,0.50,0.50,1,1,0.0,0.0,0.0,0.0,[275 65 20],1
16,275 30r20 20 inch tires extra load dunlop,0.22,0.22,1,1,0.0,0.0,0.0,0.0,[275 30r20 20],1


In [85]:
df['Tire_measures'] = df['Tire_measures'].map(lambda x: x[0])

In [86]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Tire_measures,list_counts
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,0.0,0.0,325 45r22,1
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,0.0,0.0,235 65 17,1
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,0.0,0.0,235 75 16,1
13,275 65 20 cooper discover pro,0.50,0.50,1,1,0.0,0.0,0.0,0.0,275 65 20,1
16,275 30r20 20 inch tires extra load dunlop,0.22,0.22,1,1,0.0,0.0,0.0,0.0,275 30r20 20,1


In [87]:
df['test_checking'] = df['Tire_measures'].map(lambda x: len(x))#.value_counts()

In [88]:
df['test_checking'].value_counts()

9     413615
10    152709
12     34465
11     32733
13      5712
8       4799
14      2333
15        17
16         3
Name: test_checking, dtype: int64

In [ ]:
    P_Type
    LT_Type
    No_Type
    Tire_width
    Aspect_ratio
    Construction
    Wheel_diameter

In [139]:
# Instantiating tire width and number of potential tire widths
df['tire_width'] = df['Tire_measures'].map( lambda x: re.findall(r"\d{3} ", x))
df['NUM_tire_widths'] = df['tire_width'].map( lambda x: len(x))

<ipython-input-139-887c3bdac889>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tire_width'] = df['Tire_measures'].map( lambda x: re.findall(r"\d{3} ", x))
<ipython-input-139-887c3bdac889>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NUM_tire_widths'] = df['tire_width'].map( lambda x: len(x))


In [140]:
# THESE PRESUMED MEASURE VALUES MIGHT BE INCORRECT
df['Aspect_ratio']   = df['Tire_measures'].map( 
    lambda x: int(re.findall(r" \d{2}|r\d{2}", x)[0]))

df['Wheel_diameter'] = df['Tire_measures'].map( 
    lambda x: int(re.findall(r" \d{2}|r\d{2}", x)[-1].strip('r')))


<ipython-input-140-adca7ce0d69a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Aspect_ratio']   = df['Tire_measures'].map(
<ipython-input-140-adca7ce0d69a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Wheel_diameter'] = df['Tire_measures'].map(


In [141]:
def map_funct(x):
    output = False
    if re.search(r"r", x):
        output = True
    return output
df['radial_construction'] = df['Tire_measures'].map( lambda x: map_funct(x))

<ipython-input-141-2ef955377d29>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['radial_construction'] = df['Tire_measures'].map( lambda x: map_funct(x))


In [142]:
def map_funct(x):
    output = False
    if re.search(r"p", x):
        output = True
    return output
df['Passanger_by_US'] = df['Tire_measures'].map( lambda x: map_funct(x))

<ipython-input-142-98a2974a34c7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Passanger_by_US'] = df['Tire_measures'].map( lambda x: map_funct(x))


In [143]:
def map_funct(x):
    output = False
    if re.search(r"lt", x):
        output = True
    return output
df['Light_truck'] = df['Tire_measures'].map( lambda x: map_funct(x))

<ipython-input-143-0b06c7437d9a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Light_truck'] = df['Tire_measures'].map( lambda x: map_funct(x))


In [144]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Tire_measures,list_counts,test_checking,tire_width,NUM_tire_widths,Aspect_ratio,Wheel_diameter,radial_construction,Passanger_by_US,Light_truck
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,0.0,0.0,325 45r22,1,9,[325 ],1,45,22,True,False,False
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,0.0,0.0,235 65 17,1,9,[235 ],1,65,17,False,False,False
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,0.0,0.0,235 75 16,1,9,[235 ],1,75,16,False,False,False
13,275 65 20 cooper discover pro,0.50,0.50,1,1,0.0,0.0,0.0,0.0,275 65 20,1,9,[275 ],1,65,20,False,False,False
16,275 30r20 20 inch tires extra load dunlop,0.22,0.22,1,1,0.0,0.0,0.0,0.0,275 30r20 20,1,12,[275 ],1,30,20,True,False,False


## Measure Column Cleaning

In [145]:
df = df[df['NUM_tire_widths'] == 1]

In [147]:
df['tire_width'] = df['tire_width'].map( lambda x: int(x[0].strip(' ')))

In [148]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,Tire_measures,list_counts,test_checking,tire_width,NUM_tire_widths,Aspect_ratio,Wheel_diameter,radial_construction,Passanger_by_US,Light_truck
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,0.0,0.0,325 45r22,1,9,325,1,45,22,True,False,False
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,0.0,0.0,235 65 17,1,9,235,1,65,17,False,False,False
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,0.0,0.0,235 75 16,1,9,235,1,75,16,False,False,False
13,275 65 20 cooper discover pro,0.50,0.50,1,1,0.0,0.0,0.0,0.0,275 65 20,1,9,275,1,65,20,False,False,False
16,275 30r20 20 inch tires extra load dunlop,0.22,0.22,1,1,0.0,0.0,0.0,0.0,275 30r20 20,1,12,275,1,30,20,True,False,False


In [153]:
df.drop(['Conversions', 'Conv_rate','list_counts', 'test_checking', 'NUM_tire_widths', 'Tire_measures'], 
        axis = 1, inplace = True)

In [154]:
df.head()

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conv_value,Conv_value_cost,tire_width,Aspect_ratio,Wheel_diameter,radial_construction,Passanger_by_US,Light_truck
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.0,0.0,325,45,22,True,False,False
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.0,0.0,235,65,17,False,False,False
10,bf 235 75 16,0.66,0.66,1,1,0.0,0.0,235,75,16,False,False,False
13,275 65 20 cooper discover pro,0.50,0.50,1,1,0.0,0.0,275,65,20,False,False,False
16,275 30r20 20 inch tires extra load dunlop,0.22,0.22,1,1,0.0,0.0,275,30,20,True,False,False


In [155]:
# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-z]',' ', text)
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 5 and not word in stop_words])



# Cvec, Standard
cvec = CountVectorizer(analyzer = "word",
                       min_df = 1000,
                       preprocessor = preprocess,
                       stop_words = 'english') 
# Cvec DF
df_words = pd.DataFrame(cvec.fit_transform(df['Search_term']).todense(), 
                        columns=cvec.get_feature_names())

In [156]:
df_words

,accelera,achilles,aggressive,altimax,amazon,antares,assurance,atturo,bfgoodrich,blacklion,...,uniroyal,ventus,walmart,weather,westlake,wildpeak,winter,winterforce,wrangler,yokohama
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
629013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
629014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
629015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
df_words.sum().sort_values(ascending=False).head(20)

goodyear       24378
michelin       23865
terrain        22677
cooper         21148
season         15126
walmart        14773
hankook        11953
radial         11863
firestone      10540
continental    10391
bridgestone    10116
falken          9338
pirelli         8914
wrangler        8505
yokohama        8430
winter          8331
federal         8174
goodrich        8064
country         7839
mastercraft     7673
dtype: int64

In [159]:
df.to_csv('./../G_ad_data/Current_analysis_1_31.csv', index= False)

In [160]:
df_words.to_csv('./../G_ad_data/Current_WORDS_analysis_1_31.csv', index= False)

    - Measures
         - Tire Type (P, LT or none)
         - Tire Width (first 3 numbers in millimeters)
         - Aspect Ratio (2 digit number after "/", height equal to the % of tire's width.) 
         - Construction (If R is present then the layers run radially across the tire
         - Wheel Diameter (Size of wheel the tire is supposed to fit)

p?l?t?\d{3} \d{2}.?\d{0,3} ?r? ?\d{2}
New Rows
    P_Type
    LT_Type
    No_Type
    Tire_width
    Aspect_ratio
    Construction
    Wheel_diameter

## Exploritory Data Analysis

In [6]:
# Sum of all Columns
for col in df.drop('Search_term', axis = 1).columns:
    print("=================")
    print(col)
    print(np.round(df[col].sum()))

Cost
2876546.0
Avg_cpc
738352.0
Impressions
95671871
Clicks
7761976
Conversions
60783.0
Conv_value
23596698.0
Conv_rate
1779750.0
Conv_value_cost
64754878.0


In [7]:
# Average of All Columns
for col in df.drop('Search_term', axis = 1).columns:
    print("=================")
    print(col)
    print(np.average(df[col]))

Cost
1.278633886339663
Avg_cpc
0.32819972236335226
Impressions
42.52644504630167
Clicks
3.4502225404864553
Conversions
0.02701817528802689
Conv_value
10.488805935716842
Conv_rate
0.7911042702081124
Conv_value_cost
28.783745192143126


In [8]:
df.shape

(2249703, 9)

In [9]:
df.sort_values('Conv_value_cost', ascending= False).head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
2205971,sku nito 217250,0.01,0.01,1,1,2.0,3640.00,200.0,364000.0
534102,yokohamamy507 255 70r22 5,0.01,0.01,1,1,1.0,2717.52,100.0,271752.0
642928,goodyear dyna torque r1 18.4 r42,0.01,0.01,1,1,1.0,2649.84,100.0,264984.0
724416,goodyear endurance rsa ult 245 70 19.5,0.01,0.01,1,1,1.0,1703.94,100.0,170394.0
269525,yokohama tires on sale on ebay,0.01,0.01,3,1,1.0,1583.88,100.0,158388.0
1376687,goodyear assurance cs tripletred as radial 245...,0.01,0.01,1,1,2.0,1547.36,200.0,154736.0
2088569,mu tires for truck radar,0.01,0.01,1,1,1.0,1459.64,100.0,145964.0
949490,goodyear wrangler duratrac 20 lt,0.01,0.01,1,1,1.0,1455.96,100.0,145596.0
1999892,p205 60r15 michelin,0.01,0.01,1,1,2.6,1371.81,260.0,137180.8
1212854,goodyear 11r 22.5 399 lhs,0.01,0.01,1,1,0.4,1334.14,40.0,133414.4


In [10]:
df.sort_values('Avg_cpc', ascending= False).head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
598529,goodyear all season tires 245 50 20,23.20,23.20,1,1,0.0,0.0,0.0,0.0
1706964,nitto nt421 265 50 20,16.75,16.75,1,1,0.0,0.0,0.0,0.0
1428623,goodyear eagle all season reviews,15.61,15.61,1,1,0.0,0.0,0.0,0.0
148068,2157514,15.35,15.35,4,1,0.0,0.0,0.0,0.0
1660559,tires online 285 45r22,30.22,15.11,3,2,0.0,0.0,0.0,0.0
1252634,295 40 22 all season,14.53,14.53,1,1,0.0,0.0,0.0,0.0
2157933,tires easy 255 55r20,14.39,14.39,1,1,0.0,0.0,0.0,0.0
417088,atturo az800 275 60r20xl 119v bsw,14.13,14.13,1,1,0.0,0.0,0.0,0.0
1652028,hankook 205 45r17 88v runflat optimo,14.07,14.07,1,1,0.0,0.0,0.0,0.0
904699,2008 honda pilot wheels,13.95,13.95,1,1,0.0,0.0,0.0,0.0


In [11]:
df_singles = df[df['Clicks'] == 1]

In [ ]:
df['Clicks'].value_counts()

In [16]:
df_measures = df[df['list_counts'] == 1]

In [ ]:
df_words.to_csv('./../G_ad_data/cvec_with_but_without_numbers.csv')

In [ ]:
counts = df_words.sum()

In [ ]:
df_words.sum().sort_values(ascending=False).head(20)

In [ ]:
test = counts[counts >= 1000]
test.

In [ ]:
df_words.shape

In [ ]:
df_measures[df['Clicks'] == 1]

In [ ]:
df_measures['list_counts'].value_counts()

In [ ]:
df_words[list(test.index)]

In [ ]:
df_measures

In [6]:
#df.drop(['Unnamed: 0', 'Currency', 'Impr_top_%', 'Impr_abs_top_%'], axis = 1, inplace = True)

In [ ]:
#df.to_csv('./../G_ad_data/G_ad_clean.csv', index = False)

In [ ]:
df_singles.head()

In [ ]:
df_singles['Conversions'].value_counts()